In [ ]:
import torch
import torch.nn as nn


class MLP(nn.Module):
    """
    Multi-layer Perceptron (MLP) - Feedforward Neural Network Model

    This model is composed of multiple fully connected (Linear) layers followed by non-linear activation functions
    and dropout layers to reduce overfitting. The final layer is the output layer, which returns logits and class
    probabilities.

    Arguments:
        input_dim (int): number of input features
        hidden_dim (int): number of hidden units in each fully connected layer
        num_classes (int): number of classes in the classification task
        dropout (float): dropout rate applied after each fully connected layer

    """
    def __init__(self, input_dim=1024, hidden_dim=512, num_classes=16, dropout=0.10):
        super(MLP, self).__init__()  # Inherited from the parent class nn.Module
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(p=dropout)
        )
        self.classifier = nn.Linear(hidden_dim, num_classes)

    def forward(self, batch):
        """
        Forward Propagation of the MLP Model
       """
        features = self.fc(batch)
        logits = self.classifier(features)
        probs = torch.softmax(logits, dim=-1)
        return probs

In [ ]:
device = "cpu"
if (torch.cuda.is_available()):
  device = "cuda"
print("device: " + device)

In [ ]:
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

def train(model, device, train_loader, optimizer, epoch, excluded, log_interval):
    model.train()
    total_loss = 0
    batch_count = len(train_loader)
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        probs = model(data)
        loss = F.nll_loss(input = probs, target =  target, reduction = "none")
        loss_mask = []
        for num in target:
            loss_mask.append(num not in excluded)
        loss_mask = torch.tensor(loss_mask)
        loss_masked = loss.where(loss_mask, torch.tensor(0.0))
        new_loss = loss_masked.sum() / loss_mask.sum() 
        new_loss.backward()
        optimizer.step()
        total_loss += new_loss.cpu().item()
        # if batch_idx % log_interval == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(data), len(train_loader.dataset),
        #         100. * batch_idx / len(train_loader), new_loss.item()))
    
    print(total_loss / (batch_count))
    return total_loss / (batch_count)


In [ ]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
# import random
# import numpy as np


# def set_seed(self):
#     """
#     Sets the seed for the random number generator.

#     This function sets the seed for both the NumPy and PyTorch random number generators, ensuring that results are reproducible.

#     Parameters:
#     seed (int): The seed to be set for the random number generators.

#     Returns:
#     None
#     """
#     random.seed(self.seed)
#     os.environ['PYTHONHASHSEED'] = str(self.seed)
#     np.random.seed(self.seed)
#     torch.manual_seed(self.seed)
#     if str(self.device.type) == 'cuda':
#         torch.cuda.manual_seed(self.seed)
#         torch.cuda.manual_seed_all(self.seed)  # if you are using multi-GPU.
#     torch.backends.cudnn.benchmark = False
#     torch.backends.cudnn.deterministic = True

In [ ]:

# Example NumPy arrays
X_train = np.random.rand(1000, 784)  # 1000 samples, 784 features each
y_train = np.random.randint(0, 10, 1000)  # 1000 labels for 10 classes

X_test = np.random.rand(200, 784)  # 200 test samples
y_test = np.random.randint(0, 10, 200)  # 200 test labels

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # For classification tasks

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader

# Create TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory = True)


In [ ]:

model = MLP().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=0.0001)

epochs = 50
gamma = 0.7
excluded = [16,17] 
log_interval = 64

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch, excluded, log_interval)
    test(model, device, test_loader)
    scheduler.step()

# if args.save_model:
    torch.save(model.state_dict(), "mnist_cnn.pt")

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Example NumPy arrays
X_train = np.random.rand(1000, 784)  # 1000 samples, 784 features each
y_train = np.random.randint(0, 10, 1000)  # 1000 labels for 10 classes

X_test = np.random.rand(200, 784)  # 200 test samples
y_test = np.random.randint(0, 10, 200)  # 200 test labels

# Convert NumPy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # For classification tasks

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoader
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

input_size = 784
hidden_size = 128
output_size = 10

model = MLP(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for data, labels in train_loader:
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for data, labels in test_loader:
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print(f'Accuracy: {100 * correct / total:.2f}%')

# Save the model
torch.save(model.state_dict(), 'mlp_model.pth')

# Load the model
model = MLP(input_size, hidden_size, output_size)
model.load_state_dict(torch.load('mlp_model.pth'))
model.eval()
